In [ ]:
from google.colab import files
uploaded = files.upload()

Saving preprocessed_dataset.csv to preprocessed_dataset.csv


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
#importing the required libraries to perform the ML regression for Serum cholestoral.

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from skopt.space import Categorical, Integer
from sklearn import metrics

In [ ]:
#loading the pre-processed dataset into the dataframe.

df = pd.read_csv("preprocessed_dataset.csv")
display(df.head(20))

,age,gender,chest_pain,resting_BP,serum_cholestoral,fasting_blood_sugar,resting_ecg,max_heart_rate,exercise_angina,oldpeak,slope,num_major_vessels,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0
6,62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,3
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
8,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,2
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1


In [ ]:
# Create X from DataFrame and y as Target
X_disease = df.drop(columns='target')
y = df.target

In [ ]:
#scaling the attributes of the dataset to apply the RF model.
scaler = MinMaxScaler(feature_range=(0, 1)).fit_transform(X_disease)
X = pd.DataFrame(scaler, columns=X_disease.columns)
X.describe()

,age,gender,chest_pain,resting_BP,serum_cholestoral,fasting_blood_sugar,resting_ecg,max_heart_rate,exercise_angina,oldpeak,slope,num_major_vessels,thal
count,720.000000,720.000000,720.000000,720.000000,720.000000,637.000000,720.000000,720.000000,720.000000,720.000000,513.000000,307.000000,400.000000
mean,0.487528,0.738889,0.726389,0.431706,0.316905,0.109890,0.284123,0.567362,0.337047,0.385186,0.350877,0.226927,0.496250
std,0.187627,0.439546,0.317882,0.154089,0.106513,0.312999,0.416187,0.180363,0.472372,0.120739,0.292625,0.312367,0.482447
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.346939,0.000000,0.333333,0.333333,0.246622,0.000000,0.000000,0.436620,0.000000,0.295455,0.000000,0.000000,0.000000
50%,0.510204,1.000000,1.000000,0.416667,0.306950,0.000000,0.000000,0.577465,0.000000,0.318182,0.500000,0.000000,0.750000
75%,0.612245,1.000000,1.000000,0.500000,0.367278,0.000000,0.500000,0.704225,1.000000,0.465909,0.500000,0.333333,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#splitting the dataset into test/train dataset with 75/25 split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
#applying RF model.

rf = RandomForestClassifier()
rf.fit(X_train, y_train) #training the model.

y_pred_rf = rf.predict(X_test) #predicting the target variable.

rf_accuracy = round(rf.score(X_test, y_test), 3) #calculating the accuracy of the model.
print('RandomForestClassifier accuracy : ', rf_accuracy) #printing the accuracy score.

RandomForestClassifier accuracy :  0.606


In [ ]:
#printing the value test counts.

y_test.value_counts()

,count
target,
0,84
1,53
2,26
3,11
4,6


# **Hyperparameter Tuning**.

In [ ]:
#defing the hyperparameters - GRID SEARCH.

parameters = {
    'n_estimators' : [50, 150, 500],
    'criterion' : ['gini', 'entropy', 'log_loss'],
    'max_features' : ['sqrt', 'log2']
}

In [ ]:
#performing the grid search on the RF model.
rf_cv = GridSearchCV(estimator=rf, cv=20, param_grid=parameters).fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 12 members, which is less than n_splits=20.
  warnings.warn(


In [ ]:
param_space = {
    'n_estimators': Integer(50, 500),  # integer range instead of fixed list
    'criterion': Categorical(['gini', 'entropy', 'log_loss']),
    'max_features': Categorical(['sqrt', 'log2'])
}

In [ ]:
rf = RandomForestClassifier(random_state=42)


In [ ]:
# Bayesian Optimization with Cross Validation
bayes_cv = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=32,          # number of parameter settings to try
    cv=20,              # cross-validation folds
    n_jobs=-1,          # use all cores
    scoring='accuracy', # or another metric
    random_state=42
)

In [ ]:
bayes_cv.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 12 members, which is less than n_splits=20.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 12 members, which is less than n_splits=20.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 12 members, which is less than n_splits=20.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 12 members, which is less than n_splits=20.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 12 members, which is less than n_splits=20.
  warnings.warn(
/usr/local/lib/python3.12/dist-pack

BayesSearchCV(cv=20, estimator=RandomForestClassifier(random_state=42),
              n_iter=32, n_jobs=-1, random_state=42, scoring='accuracy',
              search_spaces={'criterion': Categorical(categories=('gini', 'entropy', 'log_loss'), prior=None),
                             'max_features': Categorical(categories=('sqrt', 'log2'), prior=None),
                             'n_estimators': Integer(low=50, high=500, prior='uniform', transform='normalize')})

In [ ]:
#printing the scores after hyperparameter tuning.
print("Best Parameters:", bayes_cv.best_params_)
print("Best CV Score:", bayes_cv.best_score_)
print('Tuned hyper parameters : ', rf_cv.best_params_)
print('accuracy : ', rf_cv.best_score_)

Best Parameters: OrderedDict({'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 73})
Best CV Score: 0.6481481481481481
Tuned hyper parameters :  {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 500}
accuracy :  0.6481481481481483
